In [1]:
import pandas as pd
from COMETH import Skeleton,DynamicSkeleton
import nimblephysics as nimble
import numpy as np
import json
import time
import csv
gui = nimble.NimbleGUI()
gui.serve(8080)

# sudo lsof -iTCP:8080 -sTCP:LISTEN

GUIWebsocketServer will start serving a WebSocket server on ws://localhost:8070
Web GUI serving on http://localhost:8080


In [2]:
gui.nativeAPI().clear()
# Build skeleton
s12 = Skeleton('BODY12.xml')
# BSM
s = DynamicSkeleton(config='BODY15_constrained_3D.xml')
s.hip_correction = False

# plot xyz axis
axis_length=0.2
gui.nativeAPI().createLine(key='x_axis',points=[np.array([0,0,0]),np.array([axis_length,0,0])],color=np.array([255,0,0,1]))
gui.nativeAPI().createLine(key='y_axis',points=[np.array([0,0,0]),np.array([0,axis_length,0])],color=np.array([0,255,0,1]))
gui.nativeAPI().createLine(key='z_axis',points=[np.array([0,0,0]),np.array([0,0,axis_length])],color=np.array([0,0,255,1]))

# Read data from CSV
markers = pd.read_csv("tmp/imu/vicon_s1_acting1.csv")
imus = pd.read_csv("tmp/imu/imu_s1_acting1.csv")

Setting len to 0.1
Setting neutral pos to   0
0.7
  0
Setting len to 0.3
Setting neutral pos to    0
-0.6
   0
Setting len to 0.1
Setting neutral pos to   0
0.2
  0


In [3]:
# Build the markers dataframe with only the subset we are interested in

Rx = np.array([
    [1,0,0 ],
    [0,0,-1 ],
    [0,1,0]
])

markers.columns
markers_dict = {
'RKnee': 'right_knee',
'LWrist': 'left_wrist',
'RHip': 'right_hip',
'RShoulder': 'right_shoulder',
'LElbow': 'left_elbow',
'LHip': 'left_hip',
'RElbow': 'right_elbow',
'RWrist': 'right_wrist',
'LKnee': 'left_knee',
'LShoulder': 'left_shoulder',
'RAnkle': 'right_ankle',
'LAnkle': 'left_ankle'
 }
target = []
for i in range(markers.shape[0]):
    row = []
    for kp in markers_dict.keys():
        p = np.array([markers[markers_dict[kp]+"_x"][i],markers[markers_dict[kp]+"_y"][i],markers[markers_dict[kp]+"_z"][i]])
        # rotate the 3d point -90 on the x axis (from y up to z up)
        p_n = Rx.dot(p)
        # print(p,p_n)
        row += p_n.tolist()
    target.append(row)
target = np.array(target)


In [4]:
# Move the body model using gt to the first position
kps = list(markers_dict.keys())
s.reset()
s12.load_from_numpy(target[0,:].reshape(-1,3),s.kps)
s.load_from_BODY12(s12)
s.exact_scale()
gui.nativeAPI().renderSkeleton(s._nimble)

In [5]:
# Plot imu position with axes
def plot_imu(imu,axis_length=0.1,imu_key='imu'):
    j_w = imu[0].getWorldTransform().matrix()   # From joint to world
    imu_j = imu[1].matrix()                     # From IMU to joint
    imu_w = j_w @ imu_j # From IMU to world
    
    # Origin
    p0 = imu_w @ np.eye(4)
    gui.nativeAPI().createSphere(key=imu_key+'o', radii=np.array([0.01,0.01,0.01]),pos=p0[0:3,3],color=np.array([255,255,0,1]))
    
    # X coordinate
    px = np.eye(4)
    px[0:3,3] = [axis_length,0,0]
    px = imu_w @ px
    gui.nativeAPI().createLine(key=imu_key+'x',points=[p0[0:3,3],px[0:3,3]],color=np.array([255,0,0,1]))
    
    # Y coordinate
    py = np.eye(4)
    py[0:3,3] = [0,axis_length,0]
    py = imu_w @ py
    gui.nativeAPI().createLine(key=imu_key+'y',points=[p0[0:3,3],py[0:3,3]],color=np.array([0,255,0,1]))
    
    # Z coordinate
    pz = np.eye(4)
    pz[0:3,3] = [0,0,axis_length]
    pz = imu_w @ pz
    gui.nativeAPI().createLine(key=imu_key+'z',points=[p0[0:3,3],pz[0:3,3]],color=np.array([0,0,255,1]))

In [6]:
# Place virtual IMUs on the shoulders and thorax
from typing import List, Tuple

right_shoulder: nimble.dynamics.BodyNode = s._nimble.getBodyNode("humerus_r")
translation: np.ndarray = np.array([0.0, 0.0, 0.04])
# z is pointing up, x lateral and y frontal
rotation: np.ndarray = np.array([[1,0,0],[0,0,1],[0,-1,0]]) @ np.array([[0,1,0],[-1,0,0],[0,0,1]])
watch_offset: nimble.math.Isometry3 = nimble.math.Isometry3(rotation, translation)

left_shoulder: nimble.dynamics.BodyNode = s._nimble.getBodyNode("humerus_l")
translation: np.ndarray = np.array([0.0, 0.0, -0.04])
rotation: np.ndarray = np.array([[1,0,0],[0,0,1],[0,-1,0]]) @ np.array([[0,1,0],[-1,0,0],[0,0,1]])
watch_offset2: nimble.math.Isometry3 = nimble.math.Isometry3(rotation, translation)

thorax: nimble.dynamics.BodyNode = s._nimble.getBodyNode("thorax")
translation: np.ndarray = np.array([0.1, -0.1, 0])
rotation: np.ndarray = np.array([[1,0,0],[0,0,1],[0,-1,0]]) @ np.array([[0,1,0],[-1,0,0],[0,0,1]])
watch_offset3: nimble.math.Isometry3 = nimble.math.Isometry3(rotation, translation)

sensors: List[Tuple[nimble.dynamics.BodyNode, nimble.math.Isometry3]] = \
    [(right_shoulder, watch_offset),
     (left_shoulder, watch_offset2),
     (thorax, watch_offset3),
     ]
for i,sensor in enumerate(sensors):
    plot_imu(sensors[i],axis_length=0.05,imu_key='imu'+str(i))
    
s.IMUs = sensors

In [ ]:
# Move the body model using gt and exact qpik
kps = list(markers_dict.keys())
s.reset_history()
for i in range(target.shape[0]):
    s12.load_from_numpy(target[i,:].reshape(-1,3),s.kps)
    s.load_from_BODY12(s12)
    if i == 0:
        s.exact_scale()
    else:
        s.exact_scale(max_iterations=100)
        # s.filter([target[i,:].reshape(-1,3),])
    gui.nativeAPI().renderSkeleton(s._nimble)
    
    for j,sensor in enumerate(sensors):
        plot_imu(sensors[j],axis_length=0.05,imu_key='imu'+str(j))
    
    # 3d targets 
    for j in range(0,int(target.shape[1]/3)):
        pos = target[i,3*j:3*j+3].reshape(3,1)
        gui.nativeAPI().createSphere(key=kps[j], radii=np.array([0.01,0.01,0.01]),pos=pos,color=np.array([0,255,0,0.8]))
    

    # Plot imu
    print(s._nimble.getAccelerometerReadings(sensors))
    print(s._nimble.getGyroReadings(sensors))
    
    s.reset_history()
    time.sleep(0.01)

In [ ]:
# Move right shoulder down
s.reset()
s12.load_from_numpy(target[0,:].reshape(-1,3),s.kps)
s.load_from_BODY12(s12)
s.exact_scale()
s._nimble.setGravity(np.array([0.0, 0.0, -9.81]))
q = s._nimble.getPositions()
q[32] = 0*np.pi/180 # adduction/abduction
q[34] = 0*np.pi/180
q[33] = 20*np.pi/180
s._nimble.setPositions(q)
gui.nativeAPI().renderSkeleton(s._nimble)
acc_0 = s._nimble.getAccelerometerReadings(sensors)
for j,sensor in enumerate(sensors):
        plot_imu(sensors[j],axis_length=0.05,imu_key='imu'+str(j))
    
q_target = q.copy()
q_target[32] -= 15*np.pi/180
s._nimble.setPositions(q_target)
acc_target = s._nimble.getAccelerometerReadings(sensors)
s._nimble.setPositions(q)

print(acc_target-acc_0)

for i in range(100):
    gui.nativeAPI().renderSkeleton(s._nimble)
    for j,sensor in enumerate(sensors):
        plot_imu(sensors[j],axis_length=0.05,imu_key='imu'+str(j))
    q = s._nimble.getPositions()

    acc = s._nimble.getAccelerometerReadings(sensors)
    error = acc - acc_target
    loss = np.inner(error, error)

    d_loss_d_acc = 2 * (acc - acc_target)
    d_acc_d_joint_angles = s._nimble.getAccelerometerReadingsJacobianWrt(accs=sensors,wrt=nimble.neural.WRT_POSITION)
    d_loss_d_joint_angles = d_acc_d_joint_angles.T @ d_loss_d_acc

    d_loss_d_joint_angles[0:6] = 0.0 # Do not update the pelvis position in the worlds

    # print(np.round(d_loss_d_joint_angles))

    q -= 0.0005 * d_loss_d_joint_angles
    s._nimble.setPositions(q)
    print("loss:",loss,)

In [7]:
# Place virtual IMUs on the shoulders and thorax
s_gt = DynamicSkeleton(config='BODY15_constrained_3D.xml')
s_gt.hip_correction = False
s_gt.reset()
from typing import List, Tuple

right_shoulder: nimble.dynamics.BodyNode = s_gt._nimble.getBodyNode("humerus_r")
translation: np.ndarray = np.array([0.0, 0.0, 0.04])
# z is pointing up, x lateral and y frontal
rotation: np.ndarray = np.array([[1,0,0],[0,0,1],[0,-1,0]]) @ np.array([[0,1,0],[-1,0,0],[0,0,1]])
watch_offset: nimble.math.Isometry3 = nimble.math.Isometry3(rotation, translation)

left_shoulder: nimble.dynamics.BodyNode = s_gt._nimble.getBodyNode("humerus_l")
translation: np.ndarray = np.array([0.0, 0.0, -0.04])
rotation: np.ndarray = np.array([[1,0,0],[0,0,1],[0,-1,0]]) @ np.array([[0,1,0],[-1,0,0],[0,0,1]])
watch_offset2: nimble.math.Isometry3 = nimble.math.Isometry3(rotation, translation)

thorax: nimble.dynamics.BodyNode = s_gt._nimble.getBodyNode("thorax")
translation: np.ndarray = np.array([0.1, -0.1, 0])
rotation: np.ndarray = np.array([[1,0,0],[0,0,1],[0,-1,0]]) @ np.array([[0,1,0],[-1,0,0],[0,0,1]])
watch_offset3: nimble.math.Isometry3 = nimble.math.Isometry3(rotation, translation)

sensors: List[Tuple[nimble.dynamics.BodyNode, nimble.math.Isometry3]] = \
    [(right_shoulder, watch_offset),
     (left_shoulder, watch_offset2),
     (thorax, watch_offset3),
     ]
    
s_gt.IMUs = sensors

In [8]:
def estimate_position_from_IMUs(s,acc_target,max_iter=100,precision=0.0001):
    
    older_loss = np.inf
    for i in range(max_iter):
        q = s._nimble.getPositions()

        acc = s._nimble.getAccelerometerReadings(s.IMUs)
        error = acc - acc_target
        loss = np.inner(error, error)
        d_loss_d_acc = 2 * (acc - acc_target)
        d_acc_d_joint_angles = s._nimble.getAccelerometerReadingsJacobianWrt(accs=s.IMUs,wrt=nimble.neural.WRT_POSITION)
        d_loss_d_joint_angles = d_acc_d_joint_angles.T @ d_loss_d_acc
        d_loss_d_joint_angles[0:6] = 0.0 # Do not update the pelvis position in the worlds
        
        q -= 0.0005 * d_loss_d_joint_angles
        
        q = np.clip(q,s.q_l,s.q_u)
        
        if np.abs(older_loss - loss) < precision:
            break
        older_loss = loss
        
        s._nimble.setPositions(q)

In [ ]:
# Move the body model using accelerometers only and compare it against gt
kps = list(markers_dict.keys())

# Skeleton moved by GT
s_gt.reset()
s12.load_from_numpy(target[0,:].reshape(-1,3)+[1,0,0],s.kps)
s_gt.load_from_BODY12(s12)
s_gt.exact_scale()

# Skeleton moved by IMUs
s.reset()
s12.load_from_numpy(target[0,:].reshape(-1,3),s.kps)
s.load_from_BODY12(s12)
s.exact_scale()
s._nimble.setGravity(np.array([0.0, 0.0, -9.81]))

gui.nativeAPI().renderSkeleton(s._nimble)
gui.nativeAPI().renderSkeleton(s_gt._nimble,prefix='gt',overrideColor=np.array([0,255,0,0.8]))

q0 = s_gt._nimble.getPositions()

for i in range(1,target.shape[0]):
    s12.load_from_numpy(target[i,:].reshape(-1,3)+[1,0,0],s.kps)
    s_gt.load_from_BODY12(s12)
    
    # Move with IK
    s_gt.exact_scale(max_iterations=100,to_scale=False)
    
    # Move with accel
    acc_readings = s_gt._nimble.getAccelerometerReadings(s_gt.IMUs)
    estimate_position_from_IMUs(s,acc_target=acc_readings)
    
    
    for j,sensor in enumerate(s.IMUs):
        plot_imu(sensor,axis_length=0.05,imu_key='imu'+str(j))
    
    
    gui.nativeAPI().renderSkeleton(s._nimble)
    
    q_gt = s_gt._nimble.getPositions()
    q_gt[0:6] = q0[0:6] # Do not update the pelvis position in the worlds
    q_gt = s_gt._nimble.setPositions(q_gt)
    gui.nativeAPI().renderSkeleton(s_gt._nimble,prefix='gt',overrideColor=np.array([0,255,0,0.8]))

    s.reset_history()
    s_gt.reset_history()
    # time.sleep(0.01)

127.0.0.1 - - [16/Jul/2025 18:16:56] code 404, message File not found
127.0.0.1 - - [16/Jul/2025 18:16:56] "GET /v1/health HTTP/1.1" 404 -
127.0.0.1 - - [16/Jul/2025 18:16:57] "GET /?vscodeBrowserReqId=1752682616915 HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2025 18:17:26] code 404, message File not found
127.0.0.1 - - [16/Jul/2025 18:17:26] "GET /v1/health HTTP/1.1" 404 -
127.0.0.1 - - [16/Jul/2025 18:17:56] code 404, message File not found
127.0.0.1 - - [16/Jul/2025 18:17:56] "GET /v1/health HTTP/1.1" 404 -
127.0.0.1 - - [16/Jul/2025 18:18:26] code 404, message File not found
127.0.0.1 - - [16/Jul/2025 18:18:26] "GET /v1/health HTTP/1.1" 404 -
127.0.0.1 - - [16/Jul/2025 18:18:56] code 404, message File not found
127.0.0.1 - - [16/Jul/2025 18:18:56] "GET /v1/health HTTP/1.1" 404 -


KeyboardInterrupt: 

127.0.0.1 - - [16/Jul/2025 18:19:26] code 404, message File not found
127.0.0.1 - - [16/Jul/2025 18:19:26] "GET /v1/health HTTP/1.1" 404 -
127.0.0.1 - - [16/Jul/2025 18:19:56] code 404, message File not found
127.0.0.1 - - [16/Jul/2025 18:19:56] "GET /v1/health HTTP/1.1" 404 -
127.0.0.1 - - [16/Jul/2025 18:20:26] code 404, message File not found
127.0.0.1 - - [16/Jul/2025 18:20:26] "GET /v1/health HTTP/1.1" 404 -
